<a href="https://colab.research.google.com/github/i2mmmmm/Study/blob/main/Pytorch/Transfer%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Transfer Learning(전이 학습)
###이미 학습된 모델의 지식을 가져와서 새로운 작업에 활용하는 딥러닝 기법

###- 장점
1. 데이터 부족 문제 해결 : 작은 규모의 데이터셋에서도 효과적으로 작동할 수 있게 도와줌
2. 계산 비용 감소 : 이미 학습된 모델을 사용하여 계산 비용을 감소
3. 시간 절약 : 더 빠른 수렴을 도모하므로 학습 시간 단축

### - 두가지 방식
1. Feature Extractino(특성 추출) : 사전에 학습된 모델 일부를 가져와서 새로운 모델의 특성을 추출하는 방식, 모델의 마지막 부분에 나온 특성을 가져와서 이를 새로운 분류기에 통과시키는 방식
2. Fine-tuning(미세 조정) : 사전 학습된 모델을 가져와 일부 층을 세로운 데이터셋에 맞게 세밀하게 조정하는 방식, 일부 층 가중치를 업데이트하여 새로운 작업에 적합하게 만듬.



In [ ]:
# 오늘의 코딩은 torchvision 라이브러리에서 제공하는 ResNet 모델 가져와서
# 새로운 분류 작업을 수행하는 코드

import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms

# 사전 학습된 ResNet 모델 로딩
model = torchvision.models.resnet18(pretrained=True)

# 마지막 분류층 변경 (전이 학습)
num_classes = 10 # 예시로 10개의 클래스
model.fc = nn.Linear(model.fc.in_features, num_classes)

# 이미지 변환 및 데이터셋 로딩
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train = False, download=True, transform= transform)

train_loader = torch.utils.data.DataLoader(dataset= train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=32, shuffle=False)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 전이 학습 모델 학습
num_epochs = 5
for epoch in range(num_epochs):
  model.train()
  for images, labels in train_loader:
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
  print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# 전이 학습 모델 평가
model.eval()
correct=0
total=0
with torch.no_grad():
  for images, labels in test_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = correct/total
print(f'Accuracy on the test set: {accuracy * 100:.2f}%')






/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 114MB/s]


100%|██████████| 170498071/170498071 [00:06<00:00, 28002399.47it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
